In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import json
import os
import lyricsgenius as genius


In [2]:
#artist_list = ["ProfJam","Mike El Nite"]
artist_list = ["Valete", "Regula"]


In [3]:
load_dotenv()
geniusCreds = os.environ.get("geniusCreds")

# "(Acoustic)", "Rework", "(Live)", "(Demo)", "TBA", "FULL LENGTH", "Billboard", "BBC", "bootleg", 'فارسی', "Türkçe", "Aşk Şarkısı", "instrumental"

api = genius.Genius(geniusCreds, verbose =False, excluded_terms = ["Remix", "edit", "feat", "Paassatempo"], sleep_time=0.3, remove_section_headers = True, skip_non_songs = True)

In [4]:
def crawler(lst):
    for name in tqdm(lst):
        try:
            artist = api.search_artist(name, per_page=50)
            artist.save_lyrics()
            print(str(name) + " json saved")
        except:
            print("failed for", name)
            file = open(r"not-working.txt","a")
            file.write(name)
            file.close()

crawler(artist_list)

Wrote `Lyrics_Valete.json`
Valete json saved
Wrote `Lyrics_Regula.json`
Regula json saved



## Construção do conjunto de dados

In [5]:
#initailizing empty dataframe for concat each json info
maindata = pd.DataFrame()    

for filename in os.listdir():
    if filename.endswith(".json"): 
        with open(filename) as json_data:
           
            data = json.load(json_data)
            
            #getting artist name and release date
            songs = data.get("songs")
            songs_df = pd.DataFrame(songs)
             
            #getting artist name and release date
            songs_df["artist_name"] = data.get("name")
            songs_df['date'] = [i.get("release_date") for i in songs]

            #getting album name
            album_lst = [i.get("album") for i in songs]
            songs_df["album"] = [i.get("name") if i else "---" for i in album_lst]


            maindata = pd.concat([maindata, songs_df])
              
maindata = maindata[['artist_name', 'title', 'album', 'lyrics', 'date', 'url']]
     
maindata.to_csv("teste.csv")